## **Comparação entre as falas de Jair Bolsonaro sobre meio ambiente e a produção jornalística sobre desmatamento na Amazônia Legal**

## **Aos Fatos**

### Link para as declarações de Bolsonaro durante o governo

In [3]:
"https://www.aosfatos.org/todas-as-declara%C3%A7%C3%B5es-de-bolsonaro/"

'https://www.aosfatos.org/todas-as-declara%C3%A7%C3%B5es-de-bolsonaro/'

#### **Mount Drive**

In [4]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


### **Bibliotecas**

In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime as dt

import plotly.express as px


### **Lista de temas dos discursos do site**


In [6]:
# html
termos = ['<option value="6">Atentado</option>',
'<option value="7">Congresso</option>',
'<option value="8">Coronavírus</option>',
'<option value="9">Corrupção</option>',
'<option value="10">Cultura</option>',
'<option value="11">Defesa</option>',
'<option value="12">Direitos e Assistência Social</option>',
'<option value="13">Ditadura</option>',
'<option value="14">Economia</option>',
'<option value="15">Educação</option>',
'<option value="16">Eleições</option>',
'<option value="17">Equipe de governo</option>',
'<option value="18">Família Bolsonaro</option>',
'<option value="19">Forças Armadas</option>',
'<option value="20">Ideologia</option>',
'<option value="21">Imprensa</option>',
'<option value="22">Indígenas e quilombolas</option>',
'<option value="23">Infraestrutura</option>',
'<option value="24">Justiça</option>',
'<option value="25">Meio ambiente</option>',
'<option value="26">Outros</option>',
'<option value="27">Preconceitos</option>',
'<option value="28">Relações internacionais</option>',
'<option value="29">Saúde</option>',
'<option value="30">Segurança</option>',
'<option value="31">Turismo</option>']

# extração area e valor
lista_termos = []
for termo in termos:
    area = termo.split('">')[1].split('</')[0]
    valor = pd.Series(termo).str.extract('(\d+)', expand=False).str.lstrip().values[0]
    lista_termos.append([area, valor])

# cria df
tema_valor = pd.DataFrame(lista_termos, columns=['area','valor'])

# print
tema_valor.head(5)

,area,valor
0,Atentado,6
1,Congresso,7
2,Coronavírus,8
3,Corrupção,9
4,Cultura,10


### **Modelo para extração dos dados por mês**

In [7]:
##########################
## Declarações por tema ##
##########################

# variavel vazia
mapeamento = []
# looping
for area, numero in zip(tema_valor.area, tema_valor.valor):
    # cria o link para a raspagem
    link = f"https://www.aosfatos.org/todas-as-declara%C3%A7%C3%B5es-de-bolsonaro/api/?format=json&interval=month&theme={numero}"
    # faz a leitura do json
    base = pd.read_json(link)
    # cria a coluna do tema
    base['nome'] = f'{area}'
    # renomeia coluna
    base.rename(columns={'total':'valor',
                         'date':'data',
                         'nome':'nome'},inplace=True)
    # cria a base
    df = pd.DataFrame(base)
    # salva
    mapeamento.append(df)
    # concat
    fim = pd.concat(mapeamento, axis=0)

################################
## Declarações Gerais por mês ##
################################

# acesso o conteúdo json
total_conteudo = "https://www.aosfatos.org/todas-as-declara%C3%A7%C3%B5es-de-bolsonaro/api/?format=json&interval=month"
# leitura e armazenamento do output json
tot = pd.read_json(total_conteudo)
# cria a coluna nome com valor "geral"
tot['nome'] = 'Geral'
# renomeia as colunas
tot.rename(columns={'total':'valor',
                         'date':'data',
                         'nome':'nome'},inplace=True)
# cria dataframe
geral = pd.DataFrame(tot)
## concatena a tabela 'geral' com 'fim'
df_declarações = pd.concat([fim, geral], axis=0)


### **Processa variáveis**

In [8]:
# cria variável ano
df_declarações['ano'] = df_declarações.data.dt.year
# cria variável mes/ano
df_declarações['mes_ano'] = df_declarações['data'].dt.strftime("%Y/%m")
# filtra dados a partir do mes e ano
df_declarações = df_declarações.sort_values(by=['mes_ano'])

## salva lista
# df_declarações.to_excel('/content/gdrive/MyDrive/(2023) Pesquisa Desmatamento e Bolsonaro/banco_declaracoes_bolsonaro_mes.xlsx')
# imprime resultado
df_declarações.head()

,data,valor,nome,ano,mes_ano
0,2019-01-01,2,Atentado,2019,2019/01
0,2019-01-01,2,Cultura,2019,2019/01
0,2019-01-01,1,Defesa,2019,2019/01
0,2019-01-01,5,Direitos e Assistência Social,2019,2019/01
0,2019-01-01,6,Economia,2019,2019/01


### **Análise exploratória**

In [9]:
df_declarações.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 908 entries, 0 to 47
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   data     908 non-null    datetime64[ns]
 1   valor    908 non-null    int64         
 2   nome     908 non-null    object        
 3   ano      908 non-null    int64         
 4   mes_ano  908 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 42.6+ KB


In [10]:
df_declarações.nome.value_counts()

Geral                            48
Economia                         47
Equipe de governo                46
Eleições                         44
Congresso                        44
Meio ambiente                    42
Educação                         42
Outros                           41
Imprensa                         41
Justiça                          41
Relações internacionais          40
Indígenas e quilombolas          39
Corrupção                        38
Saúde                            37
Direitos e Assistência Social    36
Coronavírus                      35
Segurança                        34
Infraestrutura                   34
Ideologia                        34
Família Bolsonaro                29
Atentado                         28
Forças Armadas                   26
Ditadura                         22
Preconceitos                     16
Cultura                          12
Turismo                           9
Defesa                            3
Name: nome, dtype: int64

In [11]:
df_declarações.ano.value_counts()

2021    237
2020    232
2022    228
2019    211
Name: ano, dtype: int64

In [12]:
# dataset
#banco = df_declarações.loc[df_declarações['nome'].isin(['Eleições', 'Ditadura', 'Cultura', 'Congresso', 'Turismo', 'Defesa', 'Outros', 'Segurança', 'Atentado', 'Justiça', 'Preconceitos', 'Economia', 'Indígenas e quilombolas', 'Família Bolsonaro', 'Relações internacionais', 'Corrupção', 'Infraestrutura', 'Meio ambiente', 'Ideologia', 'Equipe de governo', 'Saúde', 'Coronavírus', 'Educação',  'Forças Armadas', 'Imprensa', 'Direitos e Assistência Social'])]
# Só meio ambiente
banco = df_declarações.query("nome in ['Meio ambiente']")
# tudo
#banco = df_declarações.query("nome in ['Eleições', 'Ditadura', 'Cultura', 'Congresso', 'Turismo', 'Defesa', 'Outros', 'Segurança', 'Atentado', 'Justiça', 'Preconceitos', 'Economia', 'Indígenas e quilombolas', 'Família Bolsonaro', 'Relações internacionais', 'Corrupção', 'Infraestrutura', 'Meio ambiente', 'Ideologia', 'Equipe de governo', 'Saúde', 'Coronavírus', 'Educação',  'Forças Armadas', 'Imprensa', 'Direitos e Assistência Social']")

# filtra dados por mês e ano
banco.sort_values('mes_ano', ascending=True, inplace=True)

# dados gráfico
fig = px.bar(banco,
             x=list(banco.mes_ano),
             y='valor', color="nome", barmode='stack', text_auto=True)

# formatação dos atributos do gráfico
fig.update_layout(autosize=True,
                  width = 1100,
                  height = 600,
                  template = 'plotly_white+xgridoff+ygridoff',  # utiliza modelo white e retira linhas dos eixos x e y
                  title="<i>Temáticas das declarações de Jair Bolsnaoro<br>por mês e ano <b>(2019-2022)<b><br><i><br>", # titulo do gráfico
                  #title_xanchor="auto",
                  plot_bgcolor='rgb(255, 255, 255)',
                  paper_bgcolor='rgb(255, 255, 255)',
                  xaxis_title='Mês',                          # rótulo eixo x
                  yaxis_title='Declarações (count)',          # rótulo eixo y
                  font=dict(family="arial",size=13),
                  legend_title_text='<br><br>Temas<br>',
                  legend=dict(
                      orientation="v",
                      font_family="arial"))


# formatação dos rótulos inseridos nas barras
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=True)
# formatação dos rótulos do eixo 'x'
fig.update_xaxes(tickangle = -45,rangeslider_visible=False, title_font_family="Arial")
# formatação da escala do eixo 'y'
fig.update_yaxes(range=[0,40]) ## exibe o intervalo de y a ser exibido no gráfico
# plota o gráfico
fig.show()

<ipython-input-12-6119d4677dd0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  banco.sort_values('mes_ano', ascending=True, inplace=True)


<br>
<br>

## **Dados Folha de S. Paulo**

### **Importa base Folha de S. Paulo**

In [13]:
from datetime import datetime

dados_folha = pd.read_excel('/content/gdrive/MyDrive/(2023) Observatório do PLURALISMO/(2023) pesquisa imprensa (religião)/termos_folha_pesquisa_desmatamento.xlsx')
dados_folha.head(2)

# Função para converter a string de data para datetime
def converter_data(data_str):
    # Mapeamento personalizado para os meses abreviados
    meses_dict = {'jan': '01',
                  'fev': '02',
                  'mar': '03',
                  'abr': '04',
                  'mai': '05',
                  'jun': '06',
                  'jul': '07',
                  'ago': '08',
                  'set': '09',
                  'out': '10',
                  'nov': '11',
                  'dez': '12'}

    # Dividir a string da data
    dia, mes_abreviado, ano = data_str.split('.')

    # Substituir o mês abreviado pelo número correspondente
    mes = meses_dict.get(mes_abreviado.lower(), mes_abreviado)

    # Criar a string formatada para ser usada com strptime
    data_formatada_str = f"{dia}.{mes}.{ano}"

    # Converter a string para um objeto de data e substituir discrepancias de dados
    return datetime.strptime(data_formatada_str.replace('1º','01'), "%d.%m.%Y")

# aplica função de conversão de data:
dados_folha['Data'] = dados_folha['Data'].apply(converter_data)
# cria variável mes/ano
dados_folha['Mes_Ano'] = dados_folha['Data'].dt.strftime("%Y/%m")

dados_folha['valor'] = 1

In [14]:
dados_folha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484 entries, 0 to 483
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   484 non-null    int64         
 1   Ano          484 non-null    int64         
 2   Area         484 non-null    object        
 3   Data         484 non-null    datetime64[ns]
 4   Titulo       484 non-null    object        
 5   Subtitulo    484 non-null    object        
 6   Mes_Ano      484 non-null    object        
 7   Mes          484 non-null    object        
 8   Dupli        484 non-null    bool          
 9   Termo        484 non-null    object        
 10  Termo_busca  484 non-null    object        
 11  Qtd_terms    484 non-null    int64         
 12  Link         484 non-null    object        
 13  valor        484 non-null    int64         
dtypes: bool(1), datetime64[ns](1), int64(4), object(8)
memory usage: 49.8+ KB


In [15]:
dados_folha.head(3)

,Unnamed: 0,Ano,Area,Data,Titulo,Subtitulo,Mes_Ano,Mes,Dupli,Termo,Termo_busca,Qtd_terms,Link,valor
0,0,2020,editorial,2020-09-20,Brasil em chamas,"Vale para o Pantanal e para a Amazônia, onde o...",2020/09,set,False,F,desmatamento+na+amazônia+legal,1,https://agora.folha.uol.com.br/editorial/2020/...,1
1,1,2021,?p=1019,2021-11-18,"Com salto de 22%, desmatamento anual na Amazôn...",por satélite do desmatamento na Amazônia Legal...,2021/11,nov,False,F,desmatamento+na+amazônia+legal,1,https://ambiencia.blogfolha.uol.com.br/?p=1019,1
2,2,2021,?p=1030,2021-11-23,PL no Senado pode induzir 53 mil km² de desmat...,de toneladas - equivalente à emissão de 22 ano...,2021/11,nov,False,F,desmatamento+na+amazônia+legal,1,https://ambiencia.blogfolha.uol.com.br/?p=1030,1


In [16]:
dados_folha.Ano.value_counts()

2022    131
2020    128
2021    128
2019     97
Name: Ano, dtype: int64

<br>
<br>

## **Junção das planilhas do Aos Fatos com Folha de S. Paulo**

In [17]:
# Filtrar os dados de Meio ambiente em df_declarações
banco = df_declarações.query("nome in ['Meio ambiente']")
banco.columns = banco.columns.str.replace('mes_ano','Mes_Ano').str.replace('ano','Ano')
banco = banco[['nome','valor','Mes_Ano', 'Ano']]

# Agrupa dados_folha por mês/ano
folha = dados_folha.groupby(['Mes_Ano','Ano']).valor.sum().reset_index()
folha['nome'] = 'Desmatamento'
base_comparação = pd.concat([banco, folha], ignore_index=True, axis=0)
base_comparação['Mes_Ano'] = pd.to_datetime(base_comparação['Mes_Ano'])
base_comparação = base_comparação.sort_values(['Mes_Ano','nome','valor'])
base_comparação.to_excel('teste.xlsx')

In [19]:
base_comparação.head(2)

,nome,valor,Mes_Ano,Ano
42,Desmatamento,2,2019-01-01,2019
0,Meio ambiente,4,2019-01-01,2019


<br>
<br>

## **Gráfico comparativo**

In [20]:
# Criar o gráfico com os dados de df_declarações
import plotly.graph_objects as go

Folha = base_comparação[base_comparação['nome']=='Desmatamento']
dt
Bozo = base_comparação[base_comparação['nome']=='Meio ambiente']

# Fig
fig = go.Figure()

# Add a line trace for 'Meio ambiente'
fig.add_trace(go.Scatter(x=Bozo[Bozo['nome']=='Meio ambiente']['Mes_Ano'],
                         y=Bozo[Bozo['nome']=='Meio ambiente']['valor'],
                         mode='lines+markers',
                         name='Declarações de Bolsonaro<br>sobre Meio Ambiente'))


# Add a line trace for 'Desmatamento'
fig.add_trace(go.Scatter(x=Folha[Folha['nome']=='Desmatamento']['Mes_Ano'],
                         y=Folha[Folha['nome']=='Desmatamento']['valor'],
                         mode='lines+markers',
                         name='Notícias (FSP)',connectgaps=True,
                         line=dict(width=1.5, dash='dot'),
                         marker=dict(size=4.5)))

# Update layout
fig.update_layout(title_text='Comparação entre Meio Ambiente e Desmatamento',
                  xaxis_title='Mês/Ano',
                  yaxis_title='Valor',
                  legend_title_text='Categoria')


fig.update_layout( template = 'plotly_white+xgridoff+ygridoff',
            title="<i>Declarações feitas por Bolsonaro sobre Meio Ambiente e<br>total de notícias sobre Desmatamento na Amazônia Legal<br><b>(2019-2022)<b><br><i>",
            title_xanchor="auto",
            plot_bgcolor='rgb(255, 255, 255)',
            paper_bgcolor='rgb(255, 255, 255)',
                          xaxis_title='Mês/ano',
                          yaxis_title='Total de declarações e de notícias<br>(Count)',
                          font=dict(family="arial",size=12),
                          #legend_title_text='<br><br>Temas<br>',
                          legend=dict(
                orientation="v",
                font_family="arial")
)
ticks_x = np.array(base_comparação['Mes_Ano'].dt.strftime('%Y/%b'))

fig.update_xaxes(
    tickangle=-55,
    rangeslider_visible=False,
    title_font_family="Arial")


# Exibir o gráfico
fig.show()
